# Exploring the Fama-French Momentum data-set

Momentum and Value are the most persistent "anomalies" in equity investing. But what these mean to an academic is often completely different from how these are applied in practice.

Academics call these "anomalies" because the standard CAPM/efficient-market model doesn't describe why these effects exists. They show that these effects are persistent by creating a long-short portfolio. For example, a momentum portfolio would be long the universe of stocks with “high” momentum and short the universe with “low” momentum.

However, in the real world, almost all portfolios are long-only. Also, there are constraints in terms of the universe of stocks because of liquidity, size, etc. Market beta plays a smaller role in a long-short momentum portfolio compared to the long-only portfolio. So a real-world long-only momentum portfolio’s return profile will be completely different from that of a long-short academic portfolio.

In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")

#initialize
famaFrench <- FamaFrench()

## Momentum Factor

The Fama-French momentum factor is the **average** return of two high prior return portfolios *minus* the **average** return of two low prior return portfolios. Each portfolio has a size (market-equity) element thrown into the mix.

Here's the complete description from their [website](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_mom_factor_daily.html):

```
We use six value-weight portfolios formed on size and prior (2-12) returns to construct Mom. The portfolios, which are formed daily, are the intersections of 2 portfolios formed on size (market equity, ME) and 3 portfolios formed on prior (2-12) return. The daily size breakpoint is the median NYSE market equity. The daily prior (2-12) return breakpoints are the 30th and 70th NYSE percentiles.

Mom is the average return on the two high prior return portfolios minus the average return on the two low prior return portfolios,

 	Mom = 1/2 (Small High + Big High) - 1/2(Small Low + Big Low).
```

Fama-French publish both daily and monthly returns of the Momentum Factor. We will look at each in turn.

### Daily Returns

In this data-set, the portfolio is rebalanced on a daily basis. Not very practical but it gives us an idea of how the factor behaves through time.

In [ ]:
dailyMom <- famaFrench$MomentumDaily() %>%
    filter(KEY_ID == 'MOM') %>%
    select(TIME_STAMP, RET) %>%
    collect()
    
dailyMomXts <- xts(dailyMom$RET/100, dailyMom$TIME_STAMP)

print("Momentum Returns: ")
print(head(dailyMomXts))
print(tail(dailyMomXts))

In [ ]:
plotRollingReturns <- function(numYears, retXts){
    avgDays <- as.numeric(index(retXts) %>% diff %>% mean)
    
    retUnits <- NA
    numPer <- NA
    if (avgDays < 10){
        numPer <- 220 * numYears
        retUnits <- "days"
    } else if (avgDays >= 10 && avgDays <= 35){
        numPer <- 12 * numYears
        retUnits <- "months"
    } else {
        stop('retXts is of invalid units')
    }
    
    rollRet <- rollapply(retXts, numPer, Return.cumulative)
    rollRet <- 100*na.omit(rollRet)
    names(rollRet) <- c('RET')
    
    rollRetDf <- data.frame(rollRet)
    rollRetDf$T <- index(rollRet)

    firstDate <- first(index(rollRet))
    lastDate <- last(index(rollRet))
    xAxisTicks <- seq(from=firstDate, to=lastDate, length.out=10)

    ggplot(rollRetDf, aes(x=T, y=RET)) +
        theme_economist() +
        geom_line() +
        scale_x_date(breaks = xAxisTicks) +
        labs(x='', y='cumulative return (%)', color='', 
             title=sprintf("Momentum Factor %d-year Rolling Returns (%s)", numYears, retUnits), 
             subtitle=sprintf("[%s:%s]", firstDate, lastDate)) +
        annotate("text", x=lastDate, y=min(rollRetDf$RET, na.rm=T), label = "@StockViz", 
                 hjust=1.1, vjust=-1.1, col="white", cex=6, fontface = "bold", alpha = 0.8)
}

In [ ]:
#5-year rolling returns
plotRollingReturns(5, dailyMomXts)

In [ ]:
#10-year rolling returns
plotRollingReturns(10, dailyMomXts)

A 5-year time-frame seems inadequate to benefit from momentum. There are multiple periods where the momentum factor returns have been negative and too close to zero for comfort. Note that these returns don't take transaction costs and taxes into account. Something to keep in mind while looking at pre-2000 datasets.

### Monthly Returns

Absolutely no one rebalances their portfolio every day. Most momentum portfolios stick to a monthly rebalance schedule. Let's see how that behaves through time.

In [ ]:
monthlyMom <- famaFrench$MomentumMonthly() %>%
    filter(KEY_ID == 'MOM') %>%
    select(TIME_STAMP, RET) %>%
    collect()
    
monthlyMomXts <- xts(monthlyMom$RET/100, monthlyMom$TIME_STAMP)

print("Momentum Returns: ")
print(head(monthlyMomXts))
print(tail(monthlyMomXts))

In [ ]:
#5-year rolling returns
plotRollingReturns(5, monthlyMomXts)

In [ ]:
#10-year rolling returns
plotRollingReturns(10, monthlyMomXts)

Irrespective of what period you use, the 5-year factor returns are bound to disappoint. Not that 10-years are "bulletproof." It too has periods where trailing returns disappoint. 

Most investors are long-only and the factor returns shown above are long-short. In the next part of this series, we will construct a long-only and long-short portfolio to get a sense for what the short portion of the portfolio contributes to investor returns. Stay tuned!

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)